In [ ]:
import numpy as np
import scipy
import scipy.stats
import torch as t
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

In [ ]:
%matplotlib inline

In [ ]:
if t.cuda.is_available():
    if t.cuda.device_count()>1:
        device = t.device('cuda:1')
    else:
        device = t.device('cuda')   
else:
    device = t.device('cpu')

In [ ]:
#device=t.device('cpu')

In [ ]:
angle = np.random.uniform(-np.pi,np.pi,(1000,1)).astype('float32')
data = np.concatenate((np.cos(angle), np.sin(angle)),axis=1)

In [ ]:
plt.scatter(data[:,0], data[:,1]);

In [ ]:
data_t = t.from_numpy(data)

In [ ]:
data_t = data_t.to(device)

In [ ]:
from torch.nn import Sequential, Linear, ReLU, LeakyReLU, Dropout, Sigmoid

In [ ]:
discriminator = Sequential(Linear(2,512), LeakyReLU(0.2, inplace=True),
                           Linear(512,32), LeakyReLU(0.2, inplace=True),
                                Linear(32,1),  Sigmoid()
                                     )

In [ ]:
discriminator = discriminator.to(device)

In [ ]:
d_optimizer = t.optim.Adam(discriminator.parameters(), lr=0.0008)

In [ ]:
generator = Sequential(Linear(2,512), LeakyReLU(),
                               Linear(512,256), LeakyReLU(),
                               Linear(256,128), LeakyReLU(),
                                Linear(128,64), LeakyReLU(),
                                Linear(64,32), LeakyReLU(),
                               Linear(32,2)
                                     )

In [ ]:
generator= generator.to(device)

In [ ]:
out_t = generator(t.empty(1000,2, device=device).uniform_(-1,1));

In [ ]:
plt.scatter(out_t.data.cpu().numpy()[:,0],out_t.data.cpu().numpy()[:,1])

In [ ]:
g_optimizer = t.optim.Adam(generator.parameters(), lr=0.0002)

In [ ]:
bce =t.nn.BCELoss()

In [ ]:
z_fixed = t.empty(1000,2, device=device).uniform_(-1,1)

mini_batch_size = 100
k_discriminator = 4
k_generator = 1
for epoch in range(1,501):
    for batch in range(len(data)//mini_batch_size):
        for k_d in range(k_discriminator):
            d_optimizer.zero_grad()
            kr = np.random.randint(0,len(data)//mini_batch_size )
          
            d = data_t[kr*mini_batch_size:(kr+1)*mini_batch_size]
            real_labels = t.ones(mini_batch_size, 1, device=device)
            d_real_loss = bce(discriminator(d), 0.9*real_labels)
      
            z = t.empty(mini_batch_size,2, device=device).uniform_(-1,1)
            g_out = generator(z)
            fake_labels = t.zeros(mini_batch_size, 1, device=device)
            d_fake_loss = bce(discriminator(g_out), fake_labels)
            d_loss = d_real_loss + d_fake_loss
            d_loss.backward()
            d_optimizer.step()
    
        g_optimizer.zero_grad()
        z = t.empty(mini_batch_size,2, device=device).uniform_(-1,1)
        g_out = generator(z)
        g_loss = bce(discriminator(g_out), real_labels)
        g_loss.backward()
        g_optimizer.step()
        
    if epoch%25 == 0:
        with t.no_grad():
                z = t.empty(len(data_t),2, device=device).uniform_(-1,1)
                real_labels = t.ones(len(data_t),1, device=device)
                fake_labels = t.zeros(len(data_t),1, device=device)
                g_out =  generator(z)
                dg_out = discriminator(g_out)
                d_loss =  bce(discriminator(data_t), real_labels)
                d_loss +=  bce(dg_out, fake_labels)
                
                g_loss = bce(dg_out, real_labels)
                clear_output()
                print(epoch, d_loss.item(), g_loss.item())    
                
                out_t = generator(z_fixed);
                plt.scatter(data[:,0], data[:,1], c='b')
                plt.scatter(out_t.data.cpu().numpy()[:,0],out_t.data.cpu().numpy()[:,1],
                            c='orange',s=5)
              
                plt.show()
            
   
    
    

In [ ]:
out_t = generator(t.empty(1000,2, device=device).uniform_(-1,1));
plt.scatter(out_t.data.cpu().numpy()[:,0],out_t.data.cpu().numpy()[:,1]);